In [3]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

In [4]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [6]:
# Load the review data
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Dr. Johnson explains complex topics with clarity and enthusiasm. Her classes are well-organized, and she is always willing to help students.'},
 {'professor': 'Prof. Mark Davis',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Davis is very knowledgeable but can be a bit fast-paced. Overall, his teaching style is effective, and he encourages active participation.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Computer Science',
  'stars': 3,
  'review': 'Dr. Lee is knowledgeable, but her lectures can sometimes be hard to follow. The course material is interesting, but assignments can be challenging.'},
 {'professor': 'Prof. Robert White',
  'subject': 'History',
  'stars': 2,
  'review': 'Prof. White’s lectures lack structure, and it’s difficult to stay engaged. He often digresses, making it challenging to grasp the key points.'},
 {'professor': 'Dr. Laura Smith',
  'subject': 'Chemist

In [7]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [8]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())

Upserted count: 10
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
